In [1]:
import sys
sys.path.append("..")
%load_ext autoreload

%autoreload 2
import pandas as pd
from misc.config import *
import logging
from models.strategy import *

from experiments import experiment_runner, fit_active_learning, fit_grid

from kaggle_ninja import *

from experiments.utils import *
from experiment_runner import run_experiment

import inspect
from get_data import *
inspect.getsourcelines(run_experiment)
from get_data import _get_raw_data, fingerprints, proteins
from sklearn.decomposition import RandomizedPCA
from sklearn.cluster import DBSCAN
from sklearn.manifold import MDS
from get_data import *

/usr/lib/python2.7/dist-packages/pkg_resources.py:1031: UserWarning: /home/kudkudak/.python-eggs is writable by group/others and vulnerable to attack when used with get_resource_filename. Consider a more secure location (set with .set_extraction_path or the PYTHON_EGG_CACHE environment variable).
  warnings.warn(msg, UserWarning)


In [ ]:
from experiments.experiment_runner import run_experiment, run_experiment_grid
from experiments import experiment_runner, fit_active_learning, fit_grid
from sklearn.svm import SVC
 
grid_result_passive = run_experiment("fit_grid",
                                    recalculate_experiments=True,
                                    n_jobs = 4, 
                                    experiment_detailed_name="fit_grid_passive",
                                    base_experiment="fit_active_learning",
                                    seed=666,
                                    grid_params = {"base_model_kwargs:alpha": list(np.logspace(-5,5,10))}, 
                                    base_experiment_kwargs={"strategy": "random_query",
                                                       "loader_function": "get_splitted_uniform_data",
                                                       "batch_size": 1, \
                                                       "base_model": "SGDClassifier",
                                                       "loader_args": {"n_folds": 2}})

In [ ]:
%matplotlib inline
plot_monitors([grid_results_uncert.experiments[0], grid_results.experiments[0]])

In [ ]:

loader = ["get_splitted_data",
          {"n_folds": 5,
           "seed":777,
           "test_size":0.0}]
strategy = "random_query"
f = "ExtFP"
grid_results = run_experiment("fit_grid",
                       n_jobs = 2, \
                       experiment_detailed_name="5ht6_"+f+"_"+strategy,
                       base_experiment="fit_active_learning",
                       seed=777,
                       grid_params = {"base_model_kwargs:alpha": list(np.logspace(-5,5,10))},
                       base_experiment_kwargs={"strategy": strategy,
                                               "loader_function": loader[0],
                                               "loader_args": loader[1],
                                                "protein": "5ht6",
                                                "fingerprint":f,
                                               "batch_size": 20, \
                                               "base_model": "SGDClassifier"})

In [ ]:

# preprocess_fncs = [["to_binary", {"all_below": True}]]
# loader = ["get_splitted_data_clusterly", {
#         "seed": 777, "preprocess_fncs": preprocess_fncs, "n_folds": 2}]
# data = get_data([["5ht6", "KlekFP"]], loader, preprocess_fncs, force_reload=True)

strategy = "random_query"
f = "ExtFP"
grid_results = run_experiment("fit_grid",
                       n_jobs = 2, \
                       experiment_detailed_name="5ht6_"+f+"_"+strategy,
                       base_experiment="fit_active_learning",
                       seed=777,
                       grid_params = {"base_model_kwargs:alpha": list(np.logspace(-5,5,10))},
                       base_experiment_kwargs={"strategy": strategy,
                                               "loader_function": loader[0],
                                               "loader_args": loader[1],
                                                "protein": "5ht6",
                                                "fingerprint":f,
                                               "batch_size": 20, \
                                               "base_model": "SGDClassifier"})

In [ ]:
# Has to be agnostic to labels
# Can have different clasa distribution - seems a good idea for active learning
# Finds K clusters and this defines K folds

In [ ]:
# all_combinations = [p for p in list(product(proteins, fingerprints))]

preprocess_fncs = [["to_binary", {"all_below": True}]]
loader = ["get_splitted_data_clusterly", {
        "seed": 777, "preprocess_fncs": preprocess_fncs, "n_folds": 2}]
data = get_data([["5ht6", "KlekFP"]], loader, preprocess_fncs, force_reload=True)

In [ ]:
from experiments.utils import *

In [ ]:
print_experiment_results(grid_results.experiments)

In [ ]:
%matplotlib inline

all_combinations = [p for p in list(product(proteins, fingerprints))]

# comps = all_combinations[len(proteins)+fingerprints.index("KlekFP"):len(proteins)+fingerprints.index("KlekFP")+1]
comps = all_combinations[15:16]
fig, axes = plt.subplots(1, 1)
fig.set_figwidth(10*1)
fig.set_figheight(10*1)



if hasattr(axes, "reshape"):
    axes = axes.reshape(-1)
else:
    axes = [axes]
    
for ax, comp in zip(axes, comps):
    preprocess_fncs = [["to_binary", {"all_below": True}]]
    m = DBSCAN(eps=0.01, metric='precomputed')
    
    X, K = calculate_jaccard_distance(force_reload=True, protein=comp[0], fingerprint=comp[1], \
                                               seed=777, preprocess_fncs=preprocess_fncs)
    clusters = m.fit_predict(K)
    
    
    _, Y = _get_raw_data(comp[0], comp[1])

        
    colors = ['rgb'[int(c)] for c in clusters]

    X_pca = get_PCA_of_raw_data(n_components=10, iterated_power=10, protein=comp[0], fingerprint=comp[1])
    ax.legend()
    ax.set_title("_".join(comp))
    ax.scatter(X_pca[:, 0], X_pca[:, 1], marker='o', c=colors)
    ax.legend()
# plt.savefig('foo.png')
plt.show()

In [ ]:

all_combinations = [p for p in list(product(proteins, fingerprints))]
prot_to_draw = 1 # len(proteins)
fing_to_draw = len(fingerprints)
comps = all_combinations[0:fing_to_draw*prot_to_draw]
fig, axes = plt.subplots(prot_to_draw, fing_to_draw)
fig.set_figwidth(10*fing_to_draw)
fig.set_figheight(10*prot_to_draw)


if hasattr(axes, "reshape"):
    axes = axes.reshape(-1)
else:
    axes = [axes]
    
for ax, comp in zip(axes, comps):
    X, Y = _get_raw_data(comp[0], comp[1])
    X_pca = get_PCA_of_raw_data(n_components=10, iterated_power=10, protein=comp[0], fingerprint=comp[1])
    ax.legend()
    ax.set_title("_".join(comp))
    for c, i, target_name in zip("rgb", [-1, 1], ["inactive", "active"]):
        if i == 0:
            continue
        ax.scatter(X_pca[Y == i, 0], X_pca[Y == i, 1], c=c, label=target_name)
    ax.legend()
# plt.savefig('foo.png')
plt.show()

In [ ]:
n_folds = 2

In [ ]:

protein = "5ht6"
fingerprint = "SubFP"
seed = 777
cluster_size_threshold = 0.1
preprocess_fncs = [["to_binary", {"all_below": True}]]



In [ ]:
@cached()
def get_PCA_of_raw_data(n_components, iterated_power, protein, fingerprint, only_positive=False):
    X, Y = _get_raw_data(protein, fingerprint)
    if only_positive:
        X = X[Y==1]
    model = RandomizedPCA(n_components=n_components,iterated_power=iterated_power).fit(X.toarray(), Y)
    return model.transform(X.toarray())

@cached()
def calculate_jaccard_distance(protein, fingerprint, seed, preprocess_fncs, only_positive=False):
    loader = ["get_splitted_data",
              {"n_folds": 1,
               "seed":seed,
               "test_size":0.0}]
    data = get_data([[protein, fingerprint]], loader, preprocess_fncs)
    Y = data[protein+"_"+fingerprint][0][0]["Y_train"]
    X1T = data[protein+"_"+fingerprint][0][0]["X_train"]
    if only_positive:
        X1T = X1T[Y==1]
    X2T = X1T
    X1T_sums = np.array(X1T.sum(axis=1))
    X2T_sums = np.array(X2T.sum(axis=1))
    K = X1T.dot(X2T.T)
    K = K.toarray()
    K2 = -(K.copy())
    K2 += (X1T_sums.reshape(-1,1))
    K2 += (X2T_sums.reshape(1,-1))
    K = K/K2
    return X1T, 1 - K

@cached()
def get_MDS_of_raw_data(n_components,  protein, fingerprint, seed, preprocess_fncs, only_positive=False):
    X, K = calculate_jaccard_distance(protein=protein, fingerprint=fingerprint, \
                                      seed=seed, preprocess_fncs=preprocess_fncs, only_positive=only_positive)
    print K.shape
    print X.shape
    m = MDS(n_components=n_components, n_jobs=4, dissimilarity="precomputed")
    return m, m.fit_transform(K)

In [ ]:
proteins

In [ ]:
protein, fingerprint = proteins[1], fingerprints[0]

In [ ]:
from get_data import fingerprints

In [ ]:
X_pca = get_PCA_of_raw_data(force_reload=True, \
                                 n_components=10, iterated_power=10, protein=protein, fingerprint=fingerprint, 
                            only_positive=False)  

In [ ]:
os.listdir(os.path.join(c["DATA_DIR"], "actives"))

In [ ]:
import pandas as pd

In [ ]:
help(pd.DataFrame.from_csv)

In [ ]:
from misc.config import *

In [ ]:
def calculate_jaccard_kernel(X1T, X2T):
    X1T_sums = np.array(X1T.sum(axis=1))
    X2T_sums = np.array(X2T.sum(axis=1))
    K = X1T.dot(X2T.T)
    if hasattr(K, "toarray"):
        K = K.toarray()
    K2 = -(K.copy())
    K2 += (X1T_sums.reshape(-1,1))
    K2 += (X2T_sums.reshape(1,-1))
    K = K.astype("float32")/K2
    return K

In [ ]:
n_folds = 3

In [ ]:
protein = "5ht2a"
fingerprint = "ExtFP"

In [ ]:
def calculate_jaccard_kernel(X1T, X2T):
    X1T_sums = np.array(X1T.sum(axis=1))
    X2T_sums = np.array(X2T.sum(axis=1))
    K = X1T.dot(X2T.T)
    if hasattr(K, "toarray"):
        K = K.toarray()
    K2 = -(K.copy())
    K2 += (X1T_sums.reshape(-1,1))
    K2 += (X2T_sums.reshape(1,-1))
    K = K.astype("float32")/K2
    return K

In [ ]:
# Read data
actives, inactives = get_protein_fingerprint_cluster_files(protein, fingerprint)
clusters_active = []
for a, _ in actives:
    clusters_active.append(pd.io.parsers.read_csv(os.path.join(c["DATA_DIR"], a), header=None).as_matrix())

inactive = pd.io.parsers.read_csv(os.path.join(c["DATA_DIR"], inactives), header=None).as_matrix()


In [ ]:
pairwise_distances(clusters_active[0], X[:, 0:1023], metric='l1')

In [ ]:
np.max(pairwise_distances(clusters_active[0], X[:, 0:1023], metric='l1'),axis=1)

In [ ]:
X

In [ ]:
proteins_sabina = ["5ht1a", "5ht2a", "5ht6", "5ht7", \
                   "alpha1a", "beta1", "beta2", "d1", "d2"]

In [ ]:
clusters_active[0] is clusters_active[0]

In [ ]:
protein = "beta2"
fingerprint = "KlekFP"

# Read data, not important
actives, inactives = get_protein_fingerprint_cluster_files(protein, fingerprint)

print actives, inactives

clusters_active = []
for a, _ in actives:
    clusters_active.append(pd.io.parsers.read_csv(os.path.join(c["DATA_DIR"], a), header=None).as_matrix().astype("float32"))
inactive = pd.io.parsers.read_csv(os.path.join(c["DATA_DIR"], inactives), header=None).as_matrix().astype("float32")

print "Sizes: ", [cl.shape[0] for cl in clusters_active]

# Report when found shared example
for A_id in range(len(clusters_active)):
    for B_id in range(len(clusters_active)):
        if A_id != B_id and A_id > B_id:
            print "Clusters ", A_id+1, " and ", B_id+1
            # Distances between cluster A and B using L1
            D = pairwise_distances(clusters_active[A_id], clusters_active[B_id], metric='l1')
            print "Shared examples # : ", (D==0).sum()
            print "Indices of examples: "
            for ex_id, val in enumerate(D):
                if np.min(val) == 0:
                    print "(", ex_id, np.argmin(val), ")"
                    assert all(clusters_active[A_id][ex_id] == clusters_active[B_id][np.argmin(val)])
            print "A.shape[0]=", clusters_active[A_id].shape[0], "B.shape[0]=", clusters_active[B_id].shape[0]

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances

def get_protein_fingerprint_cluster_files(protein, fingerprint):
    inactives = ["inactives/"+x for x in os.listdir(os.path.join(c["DATA_DIR"], "inactives")) if x.startswith(protein + "_inactives") and x.endswith(fingerprint+".csv")]
    actives = [("actives/"+x, int(x.split("_")[2][-1])) for x in os.listdir(os.path.join(c["DATA_DIR"], "actives")) if x.startswith(protein + "_actives") and x.endswith(fingerprint + ".csv")]
    # This magic lines ensures there are no clusters of id > 9
    assert(sorted(list(set([a[1] for a in actives]))) == sorted([a[1] for a in actives]))
    return sorted(actives, key=lambda x: x[1]), inactives[0]

def get_sabina_clusters(protein, fingerprint):

    # Read data
    actives, inactives = get_protein_fingerprint_cluster_files(protein, fingerprint)
    clusters_active = []
    for a, _ in actives:
        clusters_active.append(pd.io.parsers.read_csv(os.path.join(c["DATA_DIR"], a), header=None).as_matrix().astype("float32"))

    inactive = pd.io.parsers.read_csv(os.path.join(c["DATA_DIR"], inactives), header=None).as_matrix().astype("float32")

    # Standarize sizes
    max_cols = max(inactive.shape[1], max(cl.shape[1] for cl in clusters_active))
    for cl_id, cl in enumerate(clusters_active):
        if cl.shape[1] != max_cols:
            clusters_active[cl_id] = np.hstack([cl, np.zeros(shape=(cl.shape[0], max_cols - cl.shape[1]))])
    if inactive.shape[1] != max_cols:
            inactive = np.hstack([inactive, np.zeros(shape=(inactive.shape[0], max_cols - inactive.shape[1]))])


    # Start with biggest
    biggest_id = np.argsort([-cluster.shape[0] for cluster in clusters_active])[0]
    X = clusters_active[biggest_id]

    clusters_active_ids = [range(X.shape[0])]
    max_id = X.shape[0] - 1
    for cluster_id, cluster in enumerate(clusters_active):
        if cluster_id != biggest_id:
            if not np.isfinite(cluster).all():
                raise ValueError("F*CK, nan in cluster file.")
            similarities = np.min(pairwise_distances(cluster, X, metric='l1'), axis=1)
            X = np.vstack([X, cluster[similarities!=0]])
            start_id = max_id + 1
            existing = list(np.where(similarities==0)[0])
            clusters_active_ids.append(existing + range(start_id, start_id +  (similarities!=0).sum()))
            max_id = max(max_id, max(clusters_active_ids[-1])) # Update max_id
            assert(len(clusters_active_ids[-1]) == cluster.shape[0])
      
    # Add inactives
    X = np.vstack([X, inactive ])
    inactive_ids = range(max_id+1, max_id+1+inactive.shape[0])
    
    # Labels
    Y = np.zeros(shape=(X.shape[0], 1))
    Y[:] = 1
    Y[max_id+1:] = -1
    
    return X, Y, [np.array(cl).reshape(-1) for cl in clusters_active_ids], np.array(inactive_ids).reshape(-1)

In [ ]:
X, Y, clusters_active_ids, inactive_ids = get_sabina_clusters("5ht1a", "SubFP")

In [ ]:
def can_be_A_easier(i, X, clusters_active_ids, inactive):
    # A cluster is a good A-cluster candidate if it is big and the rest of active clusters are big
    difference_A_C = [ len(set(cl_ids).difference(clusters_active_ids[i])) for cl_ids in clusters_active_ids]
    
    active_size = X.shape[0] - inactive.shape[0]
    print difference_A_C, sum(difference_A_C), active_size
    return \
        sum(difference_A_C) > 0.1*active_size \
        and len(clusters_active_ids[i]) > 0.4*active_size

In [ ]:
def can_be_A(i):
    # A cluster is a good A-cluster candidate if there exists another cluster that has at least 10% of
    # disjoint members and is itself > 50%
    difference_A_C = [ len(set(cl_ids).difference(clusters_active_ids[i])) for cl_ids in clusters_active_ids]
    difference_C_A = [ len(set(clusters_active_ids[i]).difference(cl_ids)) for cl_ids in clusters_active_ids]
    # Idea is that at least one pair (A,C) of clusters has at least 10% unique members in A\C and C\A
    print len(clusters_active_ids[i])
    print difference_A_C
    print difference_C_A
    print 0.1 * X.shape[0]
    print [size_ac > 0.1 * X.shape[0] and size_ca > 0.06 * X.shape[0] for size_ac, size_ca in zip(difference_A_C, difference_C_A)]
    return \
        any(size_ac > 0.1 * X.shape[0] and size_ca > 0.06 * X.shape[0] 
                for size_ac, size_ca in zip(difference_A_C, difference_C_A)) \
        and len(clusters_active_ids[i]) > 0.5*X.shape[0]

In [ ]:
# Pick A cluster
s = []
sizes = []
for p in proteins_sabina:
    try:
        X, Y, clusters_active_ids, inactive_ids = get_sabina_clusters(p, "KlekFP")
#         print([cl.shape[0] for cl in clusters_active_ids])
        sizes.append([cl.shape[0] for cl in clusters_active_ids])
        A_candidates = [(cl.shape[0], id) for id, cl in enumerate(clusters_active_ids) if can_be_A_easier(id, X, clusters_active_ids, inactive_ids)]

        if len(A_candidates) == 0:
            print "Not satisfied for ", p
            s.append("Not satisfied")
        else:
            print "OK"
            s.append("OK")
    except Exception, e:
        print e
        sizes.append([])
        s.append(str(e))
#         raise ValueError("No cluster qualified for experiment A-type cluster. F*CK")
#     np.max(A_candidates)

In [ ]:
s

In [ ]:
for p, result, size in zip(proteins_sabina, s, sizes):
    print "Protein: ",p
    print "Satisfied?", result
    print "Sizes of clusters", size
    print "-----\n"

In [ ]:
pd.DataFrame(s)

In [ ]:


# Keep track of indices
size = sum(c.shape[0] for c in clusters_active) + inactive.shape[0]
cluster_id = np.zeros(shape=(size,1))
start_id = 0
for c_id, cluster in enumerate(clusters_active):
    cluster_id[start_id:start_id +cluster.shape[0]] = c_id
    start_id += cluster.shape[0]
cluster_id[start_id:] = -1

In [ ]:
# Assert that we have sensible indexing
assert(all(list(cluster_id[clusters_active[0].shape[0]: clusters_active[0].shape[0] + clusters_active[1].shape[0]] == 1)))

In [ ]:
clusters_active[0].shape[0]

In [ ]:
# Now we pick biggest one denoted as A
A_id = np.argsort([-cluster.shape[0] for cluster in clusters_active])[0]

In [ ]:
similarities = calculate_jaccard_kernel(clusters_active[0], clusters_active[A_id])

In [ ]:
B

In [ ]:
# Check if random picked

In [ ]:
from models.strategy import jaccard_similarity_score_fast

# Some magic: assigning each inactive to the closest active cluster by mean
similarities = []
for cluster in clusters_active:
    similarities.append(calculate_jaccard_kernel(inactive, cluster).mean(axis=1).reshape(-1,1))
# Each col has mean distance of col-th cluster to row-th inactive example
similarities = np.hstack(similarities) 

# Do assignment
active_size = sum(cluster.shape[0] for cluster in clusters_active)
for i in range(inactive.shape[0]):
    cluster_id[i + active_size] = np.argmax(similarities[i,:])
    
# Assert that similarities is what we want
assert np.abs(similarities[10,1] - np.mean([jaccard_similarity_score_fast(inactive[10], clusters_active[1][i]) for i in range(clusters_active[1].shape[0])])) < 1e-3    

In [ ]:
[cluster.shape[0] for cluster in clusters_active]

In [ ]:
K = calculate_jaccard_kernel(clusters_active[0], clusters_active[1])

In [ ]:
clusters_active[1]

In [ ]:
if n_folds == 1:
    fold_indices = [[range(y.shape[0]), None]]
else:
    fold_indices = StratifiedKFold(cluster_id, n_folds=n_folds, shuffle=True, random_state=seed)


In [ ]:
distances

In [ ]:
distances[1]

In [ ]:
size

In [ ]:
clusters_active[0].shape

In [ ]:
clusters_active[1].shape

In [ ]:
x = 

In [ ]:
x

In [ ]:
import seaborn
%matplotlib inline
plt.figure(figsize=(15,15))
# f, ax = plt.subplots(1,1)
# for j, cluster in enumerate(clusters[0:2]):
#     X_pca_picked = X_pca[colors==j,:]
#     cols = np.array([j for k in range(X_pca_picked.shape[0])])
plt.scatter(X_pca[:,0], X_pca[:,1], s=40, alpha=0.9, cmap=plt.get_cmap("Pastel1"))
plt.show()

In [ ]:
interestingness_index(X, big_subtrees[0], big_subtrees[1])

In [ ]:
len(set(big_subtrees[0]))

In [ ]:
interestingness_index(X, big_subtrees[0], big_subtrees[1])

In [ ]:
len(set(big_subtrees[0]).intersection(set(big_subtrees[1])))

In [ ]:
sum(len(subtree) > check_threshold for subtree in id_to_nodes.values())

In [ ]:
import graphviz as gv
g = gv.Graph(format='svg')
for i in range(K.shape[0]):
    g.node(str(i))
# Nodes in m.children_ are sorted by merging time
for id, n in enumerate(m.children_):
    g.node(str(id + K.shape[0]))
    g.edge(str(id + K.shape[0]), str(n[0]))
    g.edge(str(id + K.shape[0]), str(n[1]))
    id_to_nodes[id  + K.shape[0]] = id_to_nodes[n[0]] + id_to_nodes[n[1]]
g.render("clustering")

In [ ]:
np.where(clusters==0)

In [ ]:
@cached()
def get_PCA_of_raw_data(n_components, iterated_power, protein, fingerprint, only_positive=False):
    X, Y = _get_raw_data(protein, fingerprint)
    if only_positive:
        X = X[Y==1]
    model = RandomizedPCA(n_components=n_components,iterated_power=iterated_power).fit(X.toarray(), Y)
    return model.transform(X.toarray())



@cached()
def get_MDS_of_raw_data(n_components,  protein, fingerprint, seed, preprocess_fncs, only_positive=False):
    X, K = calculate_jaccard_distance(protein=protein, fingerprint=fingerprint, \
                                      seed=seed, preprocess_fncs=preprocess_fncs, only_positive=only_positive)
    print K.shape
    print X.shape
    m = MDS(n_components=n_components, n_jobs=4, dissimilarity="precomputed")
    return m, m.fit_transform(K)

In [ ]:
%matplotlib inline

In [ ]:
import seaborn
all_combinations = [p for p in list(product(proteins, fingerprints))]
prot_to_draw = 1 #len(proteins)
fing_to_draw = len(fingerprints)
comps = all_combinations[len(fingerprints): len(fingerprints)*2]
fig, axes = plt.subplots(prot_to_draw, fing_to_draw)
fig.set_figwidth(20*fing_to_draw)
fig.set_figheight(20*prot_to_draw)
preprocess_fncs = [['to_binary', {'all_below': True}]]


if hasattr(axes, "reshape"):
    axes = axes.reshape(-1)
else:
    axes = [axes]
    
for ax, comp in zip(axes, comps):
    print comp
#     X, Y = _get_raw_data(comp[0], comp[1])
    X_mds = get_MDS_of_raw_data(n_components=2, seed=777, protein=comp[0], \
                                fingerprint=comp[1], preprocess_fncs=preprocess_fncs, only_positive=True)
#     X_mds = get_PCA_of_raw_data(force_reload=True, \
#                                 n_components=10, iterated_power=10, protein=comp[0], fingerprint=comp[1])  
    ax.legend()
    ax.set_title("_".join(comp))
    ax.scatter(X_mds[:, 0], X_mds[:, 1])
    ax.legend()
plt.savefig('pca_5ht6_one_class_2_comp.png')
#plt.show()

In [ ]:
import seaborn
all_combinations = [p for p in list(product(proteins, fingerprints))]
prot_to_draw = 1 #len(proteins)
fing_to_draw = len(fingerprints)
comps = all_combinations[0:fing_to_draw*prot_to_draw]
fig, axes = plt.subplots(prot_to_draw, fing_to_draw)
fig.set_figwidth(10*fing_to_draw)
fig.set_figheight(10*prot_to_draw)

if hasattr(axes, "reshape"):
    axes = axes.reshape(-1)
else:
    axes = [axes]
    
for ax, comp in zip(axes, comps):
    X, Y = _get_raw_data(comp[0], comp[1])
    X = X[Y==1] # Pick only positive
    X_pca = get_PCA_of_raw_data(force_reload=True, \
                                n_components=10, iterated_power=10, protein=comp[0], fingerprint=comp[1])
    ax.legend()
    ax.set_title("_".join(comp))
    ax.scatter(X_pca[:, 0], X_pca[:, 1])
    ax.legend()
plt.savefig('foo.png')
# plt.show()

In [ ]:
get_best(grid_results_random.experiments, metric="mean_mcc_valid").results['mean_mcc_valid']

In [ ]:
get_best(grid_results_uncert.experiments, metric="mean_mcc_valid").results['mean_mcc_valid']

In [ ]:
import sklearn